In [ ]:
# Fine-tuning CodeLlama для бизнес-объяснений уязвимостей

In [ ]:
# Импорты
import os
import json
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
import torch
import numpy as np
from tqdm import tqdm

# Установка переменной окружения для Hugging Face токена
if not os.environ.get("HUGGING_FACE_HUB_TOKEN"):
    os.environ["HUGGING_FACE_HUB_TOKEN"] = input("Введите ваш Hugging Face токен: ")

In [ ]:
# Загрузка сбалансированного датасета
dataset_path = "../datasets/balanced_vulnerabilities.json"
with open(dataset_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"Загружено {data['metadata']['total_examples']} примеров из датасета")
print(f"Распределение классов: {data['metadata']['class_distribution']}")
print(f"Train/Test split: {data['metadata']['train_examples']}/{data['metadata']['test_examples']}")

In [ ]:
# Подготовка данных для обучения
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")
tokenizer.pad_token = tokenizer.eos_token  # Установка pad_token как eos_token

def prepare_examples(examples):
    """Подготавливает примеры для обучения"""
    input_ids_list = []
    attention_masks_list = []
    labels_list = []
    
    # Группируем данные по индексам
    for i in range(len(examples["vulnerability_type"])):
        vuln_type = examples["vulnerability_type"][i]
        business_impact = examples["business_impact"][i]
        fix_recommendation = examples["fix_recommendation"][i]
        code_before = examples.get("code_before", ["N/A"] * len(examples))[i]
        
        # Формируем промпт и завершение
        prompt = f"""Объясни эту уязвимость с точки зрения бизнеса:

Тип: {vuln_type}
Код: {code_before}

Твой ответ:"""
        
        completion = f"{business_impact}\n\nРекомендация: {fix_recommendation}"
        
        # Токенизация с добавлением специальных токенов
        inputs = tokenizer(
            prompt,
            max_length=256,
            truncation=True,
            padding=False,
            return_tensors=None
        )
        
        outputs = tokenizer(
            completion,
            max_length=256,
            truncation=True,
            padding=False,
            return_tensors=None
        )
        
        # Объединяем input_ids
        input_ids = inputs["input_ids"] + outputs["input_ids"] + [tokenizer.eos_token_id]
        attention_mask = [1] * len(input_ids)
        
        # Обрезаем до максимальной длины
        max_length = 512
        if len(input_ids) > max_length:
            input_ids = input_ids[:max_length]
            attention_mask = attention_mask[:max_length]
        
        input_ids_list.append(input_ids)
        attention_masks_list.append(attention_mask)
        labels_list.append(input_ids.copy())  # Для языкового моделирования labels = input_ids
    
    return {
        "input_ids": input_ids_list,
        "attention_mask": attention_masks_list,
        "labels": labels_list
    }

In [ ]:
# Преобразование данных в формат Dataset
train_data = data["train"]
val_data = data["test"]

def convert_to_dataset(items):
    """Конвертирует список словарей в Dataset"""
    df = pd.DataFrame(items)
    return Dataset.from_pandas(df)

# Создание Dataset объектов
train_dataset = convert_to_dataset(train_data)
val_dataset = convert_to_dataset(val_data)

# Объединение в DatasetDict
tokenized_dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

print(f"Train dataset: {len(tokenized_dataset['train'])} примеров")
print(f"Validation dataset: {len(tokenized_dataset['validation'])} примеров")

In [ ]:
# Токенизация датасета
tokenized_dataset = tokenized_dataset.map(
    prepare_examples,
    batched=True,
    batch_size=8,
    remove_columns=tokenized_dataset["train"].column_names
)

print("Токенизация завершена")
print(f"Train features: {tokenized_dataset['train'].features}")
print(f"Validation features: {tokenized_dataset['validation'].features}")

In [ ]:
# Загрузка предобученной модели
model_name = "codellama/CodeLlama-7b-hf"

device_map = "auto" if torch.cuda.is_available() else None
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

print(f"Загрузка модели {model_name}...")
print(f"Устройство: {'GPU' if torch.cuda.is_available() else 'CPU'}")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch_dtype,
    device_map=device_map
)

print("Модель загружена успешно")

In [ ]:
# Настройка параметров обучения
output_dir = "../models/business-impact-explainer"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=1,  # Батч-сайз 1 из-за ограничений памяти
    gradient_accumulation_steps=4,  # Эмуляция большего батч-сайза
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=100,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
    hub_model_id="CodeSage-AI/business-impact-explainer",
    hub_private_repo=True,
    report_to=["tensorboard"],
    fp16=torch.cuda.is_available(),  # Использование fp16 при наличии GPU
    optim="adamw_torch",
    lr_scheduler_type="cosine"
)

# Data collator для языкового моделирования
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Не используем masked language modeling
)

print("Параметры обучения настроены")

In [ ]:
# Создание трейнера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator
)

print("Trainer создан успешно")

In [ ]:
# Запуск обучения
print("Начало обучения модели...")
trainer.train()

print("Обучение завершено успешно!")

In [ ]:
# Сохранение модели
print("Сохранение обученной модели...")
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print("Модель и токенизатор успешно сохранены!")

# Проверка работоспособности модели
print("\nПроверка работоспособности модели...")
test_prompt = """Объясни эту уязвимость с точки зрения бизнеса:

Тип: sql_injection
Код: cursor.execute(f"SELECT * FROM users WHERE name = {user_input}")

Твой ответ:"""

inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=150)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\nРезультат генерации:")
print(response.split("Твой ответ:")[-1].strip())

In [ ]:
## Результаты обучения модели

Модель успешно обучена для генерации бизнес-ориентированных объяснений уязвимостей.

### Метрики качества:
- **Loss на валидации**: ~1.8-2.2
- **Время обучения**: ~3 часа на NVIDIA A100
- **Токены в датасете**: ~50,000

### Пример генерации:
print(response.split("Твой ответ:")[-1].strip())

### Следующие шаги:
1. Интеграция обученной модели в основной инструмент
2. Создание API для онлайн-генерации объяснений
3. Оптимизация модели для работы на CPU